This script is used to create the schema and tables, and upload staged data to database.

In [18]:
from database_utils import db_connect
from constants import *

In [19]:
# try to establish the connection
db_conn, db_cur = db_connect()
# verify the connection
db_cur.execute('SELECT version()')
db_version = db_cur.fetchone()
print(db_version)

Connecting to the PostgreSQL database...
('PostgreSQL 11.11 on amd64-portbld-freebsd11.4, compiled by FreeBSD clang version 10.0.0 (git@github.com:llvm/llvm-project.git llvmorg-10.0.0-0-gd32170dbd5b), 64-bit',)


In [20]:
# create the tables by executing init.sql
db_cur.execute(open("init.sql", "r").read())
db_conn.commit()

In [21]:
# upload staged data
db_cur.execute(f"SET search_path = {SCHEMA_NAME};")

In [22]:
# close the cursor
if db_cur is not None:
    db_cur.close()

# close the connection
if db_conn is not None:
    db_conn.close()